<a href="https://colab.research.google.com/github/candiceevemiller/personal-projects/blob/main/Wordle_Cheat_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
A program that takes your wordle clues and gives you potential matches from the
official wordle answer bank
"""

import re

In [2]:
# Loads wordle answers and stores in list
!wget https://raw.githubusercontent.com/a-vanderheiden/WordleLetterDistribution/main/WordleAnswers.txt

wordle_answers = []
with open('WordleAnswers.txt', 'r') as f:
    for line in f:
        for word in line[1:-1].split(','):
            wordle_answers.append(word.strip('"'))
wordle_answers.sort()
print(f'Number of possible answers: {len(wordle_answers)}\n')
wordle_answers[:25]

--2022-02-05 23:45:01--  https://raw.githubusercontent.com/a-vanderheiden/WordleLetterDistribution/main/WordleAnswers.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18521 (18K) [text/plain]
Saving to: ‘WordleAnswers.txt’

WordleAnswers.txt   100%[===================>]  18.09K  --.-KB/s    in 0s      

2022-02-05 23:45:01 (44.4 MB/s) - ‘WordleAnswers.txt’ saved [18521/18521]

Number of possible answers: 2315



['aback',
 'abase',
 'abate',
 'abbey',
 'abbot',
 'abhor',
 'abide',
 'abled',
 'abode',
 'abort',
 'about',
 'above',
 'abuse',
 'abyss',
 'acorn',
 'acrid',
 'actor',
 'acute',
 'adage',
 'adapt',
 'adept',
 'admin',
 'admit',
 'adobe',
 'adopt']

In [3]:
def handle_duplicate_letters(green: list, yellow: list, gray: str) -> set:
    """
    In wordle doubled letters that are only matched once can return one green
    or yellow and the other gray. In order to handle this we convert our clues
    to sets and find the difference of them so that only the letters that are
    only in gray will filter our possible answers.
    """

    green_set = set(green)
    yellow_set = set(yellow)
    gray_set = set(gray)
    gray = (gray_set - yellow_set) - green_set
    return gray

In [4]:
# TODO: Refactor control flows to better use filters & generators & use all func

def potential_answers(green: list, yellow: list, gray: str) -> list:
    """
    Takes 2 lists, green, yellow, and a string gray
    example expected format:
    green = ['a', '', '', '', '', 'e']
    yellow = ['', 'ok', '', '', '']
    gray = 'brlm'
    """

    gray = handle_duplicate_letters(green, yellow, gray)

    search_sequence = ['.' for i in range(5)]  # create list equivalent to string '.....'

    # iterate over green list and if the value isn't empty replace the character
    # at the matching index in search_sequence
    for key, val in enumerate(green):
        if val:
            search_sequence[key] = val

    # create an empty string to store letters we know are in the word but don't
    # know where then we iterate over yellow list and create a regex NOT term
    # for each non-empty index and append the relavent terms to the in_word
    # variable this way we know to look for words that have a letter but not at
    # the Xth index
    in_word = ''
    for key, val in enumerate(yellow):
        in_word += val
        if val and search_sequence[key] == '.':
            search_sequence[key] = '[^' + val + ']'
    search_sequence = ''.join(search_sequence)
    r = re.compile(search_sequence)

    # filters based on green known letters and which letters aren't at an index
    # but doesn't enforce the presence of yellow letters
    regex_matches = list(filter(r.match, wordle_answers))

    # iterate through each word in regex_matches, then iterates through every
    # letter we know must be in the word, if all letters are in the word we
    # append the word to our yellow_and_green_matches list
    yellow_and_green_matches = []
    for word in regex_matches:
        flag = True
        for letter in in_word:
            if letter not in word:
                flag = False
        if flag:
            yellow_and_green_matches.append(word)

    # iterate through every word in our yellow_and_green_matches list and then
    # iterate through every letter we know NOT to be in the word. If it contains
    # none of the gray letters we append our word to our final output list
    final_matches = []
    for word in yellow_and_green_matches:
        flag = True
        for letter in gray:
            if letter in word:
                flag = False
        if flag:
            final_matches.append(word)
            
    print('\nYour potential matches are:\n')
    return final_matches

In [5]:
# Test case with actual guesses for wordle 227
green = ['', '', '', 's', 'e']
yellow = ['s', 'et', '', '', '']
gray = 'paryls'
potential_answers(green, yellow, gray)


Your potential matches are:



['these', 'those']

In [6]:
def get_green() -> list:
    """
    Gets and validates user input for green letters and formats for feeding into
    the function potential_answers()
    """

    green = ['']*5 # initialize green list

    print("\nLet's get your green letters:")
    print("-----------------------------")

    for i in range(5):
        flag = ''  # helps us validate user, keeping us in the while loop if not y/n        

        while flag != 'y' and flag != 'n':
            flag = input(f'Do you know what letter is in spot {i+1}? (y/n)\n')
            if flag != 'y' and flag !='n':
                print('Please only type y or n right now\n')

        if flag == 'y':
            letter = ''
            while not letter.isalpha() or len(letter) != 1:
                letter = input(f'What letter do you have for spot {i+1}?\n')
                if not letter.isalpha() or len(letter) != 1:
                    print('Please only type a letter right now')
            # store standardized user input in the correct index of green dict
            green[i] = letter.lower()

    return green

In [7]:
def get_yellow() -> list:
    """
    Gets and validates user input for yellow letters and formats for passing into
    function potential_answers()
    """

    yellow = ['']*5  # initialize yellow list

    print("\nLet's get your yellow letters:")
    print("-----------------------------")

    for i in range(5):
        flag = '' # again helps us validate user input

        while flag != 'y' and flag != 'n':
            flag = input(f'Do you know any yellow letters for spot {i+1}? (y/n)\n')
            if flag != 'y' and flag !='n':
                print('Please only type y or n right now\n')

        if flag == 'y':
            letters = ''
            while not letters.isalpha():
                letters = input(f'What yellow letters do you know for spot {i+1}?\n(Type all that apply)\n')
                if not letters.isalpha():
                    print('Please only type a letter right now')
            # stores standardized user input in correct index of yellow dict
            yellow[i] = letters.lower()
            
    return yellow

In [8]:
def get_gray() -> str:
    """
    Gets and validates user input for gray letters and formats for passing into
    function potential_answers
    """

    gray = ''

    print(f"\nLet's get your gray letters:")
    print('-----------------------------')

    do_you_know_gray_letters = ''
    while do_you_know_gray_letters != 'y' and do_you_know_gray_letters != 'n':
        do_you_know_gray_letters = input('Do you know any gray letters? (y/n)\n')
    
    if do_you_know_gray_letters == 'y':
        while not gray and not gray.isalpha():
            gray = input(f'What are your gray letters?\n(Type all that apply)\n')
            if not gray.isalpha() and gray != '':
                print('Please only type letters')
            
    return gray.lower()

In [9]:
def get_answers():
    """Combines input functions and output function into one"""

    green = get_green()
    yellow = get_yellow()
    gray = get_gray()

    return potential_answers(green, yellow, gray)

In [10]:
get_answers()


Let's get your green letters:
-----------------------------
Do you know what letter is in spot 1? (y/n)
n
Do you know what letter is in spot 2? (y/n)
y
What letter do you have for spot 2?
l
Do you know what letter is in spot 3? (y/n)
n
Do you know what letter is in spot 4? (y/n)
n
Do you know what letter is in spot 5? (y/n)
n

Let's get your yellow letters:
-----------------------------
Do you know any yellow letters for spot 1? (y/n)
y
What yellow letters do you know for spot 1?
(Type all that apply)
f
Do you know any yellow letters for spot 2? (y/n)
n
Do you know any yellow letters for spot 3? (y/n)
y
What yellow letters do you know for spot 3?
(Type all that apply)
a
Do you know any yellow letters for spot 4? (y/n)
n
Do you know any yellow letters for spot 5? (y/n)
n

Let's get your gray letters:
-----------------------------
Do you know any gray letters? (y/n)
persck
Do you know any gray letters? (y/n)
y
What are your gray letters?
(Type all that apply)
persck

Your potential matc

['aloft', 'aloof']